<a href="https://colab.research.google.com/github/RyanTahnikoyev/windows-97-robidatathon/blob/main/Merged_Pipeline(pre_EDA)_X_2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [92]:
import os

import pandas as pd
import numpy as np

# plotting packages
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as clrs
import seaborn as sns
import plotly
import plotly.graph_objs as go
import plotly.express as px
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score



train_data = pd.read_csv("/content/drive/MyDrive/robi-datathon-2-pre-assessment/train.csv")
# check the raw data
print("Size of the train dataset (train_data, col): ", train_data.shape)

test_data = pd.read_csv("/content/drive/MyDrive/robi-datathon-2-pre-assessment/test.csv")
# check the raw data
print("Size of the test dataset (test_data, col): ", test_data.shape)

Size of the train dataset (train_data, col):  (28322, 36)
Size of the test dataset (test_data, col):  (85065, 35)


In [93]:
def preprocessing(train_data):

    train_data[train_data["s52"]=="o"]=int("0")
    train_data[train_data["s52"]=="l"]=int("1")

    train_data = train_data.drop(['s56','s57','s59'], axis=1)
    train_data.head()

    train_data = train_data.reset_index()
    X_train_full = train_data.copy()

    binary_cols= [cname for cname in X_train_full.columns if
              X_train_full[cname].nunique()==2 and cname!=X_train_full.columns[-1]]

    # "Cardinality" means the number of unique values in a column
    # Select categorical columns with relatively low cardinality (convenient but arbitrary)
    categorical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].nunique() < 10 and 
                X_train_full[cname].dtype == "object" and cname 
                not in binary_cols]

    # Select numerical columns
    numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

    binary_df= train_data[binary_cols].copy()

    categorical_df=train_data[categorical_cols].copy()

    from sklearn.preprocessing import LabelEncoder
    for i in binary_df:
        le=LabelEncoder()
        binary_df[i]=le.fit_transform(binary_df[i])

    categorical_df=pd.get_dummies(categorical_df)
    numerical_df = train_data[numerical_cols].copy()

    processed_df = pd.concat([binary_df,categorical_df,numerical_df],axis=1)
    return processed_df

In [97]:
# Remove rows with missing target, separate target from predictors
# train_data.dropna(axis=0, subset=['label'], inplace=True)
y = train_data.label
processed_data = preprocessing(train_data)
processed_data = processed_data.drop(['label'], axis=1)
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(processed_data, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [98]:
# classifier = RandomForestClassifier(n_estimators = 100, random_state = 0)
classifier = RandomForestClassifier(n_estimators=50, criterion='gini', min_samples_split=5, min_samples_leaf=2, max_features='auto', bootstrap=True, n_jobs=-1, random_state=42)
# classifier = XGBClassifier(**xgb_params, n_estimators=1000,tree_method='gpu_hist')

# Preprocessing of training data, fit model 
classifier.fit(X_train_full, y_train)

classifier
# Preprocessing of validation data, get predictions
preds = classifier.predict(X_valid_full)

In [99]:
from sklearn.metrics import accuracy_score
accuracy_score(preds, y_valid)

0.9433362753751103

In [71]:
from sklearn.model_selection import KFold
classifier = RandomForestClassifier(n_estimators = 100, random_state = 0)
kfold_validation = KFold(10)
kfold_validation

KFold(n_splits=10, random_state=None, shuffle=False)

In [72]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(classifier,processed_data,y,cv=kfold_validation)
print(results)

[0.93399223 0.93822803 0.94456215 0.9420904  0.9325565  0.93961864
 0.94668079 0.94420904 0.9420904  0.94173729]


In [45]:
from sklearn.model_selection import StratifiedKFold
skfold= StratifiedKFold(n_splits=5)
classifier = RandomForestClassifier(n_estimators = 50, random_state = 42)
scores = cross_val_score(classifier,processed_data,y,cv=skfold)
print(results)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
for train_index, test_index in kf.split(processed_data):
    X_train_full, X_valid_full, y_train, y_valid = processed_data[train_index], processed_data[test_index], y[train_index], y[test_index]
    # classifier = RandomForestClassifier(n_estimators=50, criterion='gini', min_samples_split=5, min_samples_leaf=2, max_features='auto', bootstrap=True, n_jobs=-1, random_state=42)
    classifier = RandomForestClassifier(n_estimators = 50, random_state = 42)
    # classifier = XGBClassifier(**xgb_params, n_estimators=1000,tree_method='gpu_hist')

    # Preprocessing of training data, fit model 
    classifier.fit(X_train_full, y_train)

    # Preprocessing of validation data, get predictions
    preds = classifier.predict(X_valid_full)
    acc = accuracy_score(preds, y_valid)
    print("Accuracy Score:",acc)

In [102]:
processed_test_data = preprocessing(test_data)
processed_test_data.head()

,s13,s48,n12,n13,gender_0,gender_1,gender_F,gender_M,s11_0,s11_1,...,n6,n7,n8,n9,n10,n11,n12,n13,n14,n15
0,1,1,1,1,0,1,0,0,0,1,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1,1,1.000000,1
1,1,1,1,1,0,1,0,0,0,1,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1,1,1.000000,1
2,1,1,1,1,0,1,0,0,0,1,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1,1,1.000000,1
3,1,0,0,0,0,0,0,1,0,0,...,0.015684,-9.178200,1.675028,6.354938,5.427266,1.792147,0,0,0.003492,1
4,1,1,0,0,0,0,0,1,0,0,...,0.006406,-8.726633,1.963535,6.810158,11.487845,1.880992,0,0,0.060614,0


In [103]:
classifier

RandomForestClassifier(min_samples_leaf=2, min_samples_split=5, n_estimators=50,
                       n_jobs=-1, random_state=42)

In [104]:
preds = classifier.predict(processed_test_data)

In [107]:
len(preds)

85065

In [106]:
test_data.id

0                                                        1
1                                                        1
2                                                        1
3        b'gAAAAABinOimitAnqlgOcqnD_LeNL3WEbXNGvjd3QVPi...
4        b'gAAAAABinOi3W9p3Oka5MV_dc2TeorZUcIWOnnODSx7E...
                               ...                        
85060                                                    1
85061                                                    1
85062    b'gAAAAABinOi31zWSlD0OMhbBd3_weh7Kq6aPeO4yYqns...
85063                                                    1
85064    b'gAAAAABinOiiLo4KNZVgClHgtOFRzEU9O97My6MowJFa...
Name: id, Length: 85065, dtype: object

In [114]:
# Save test predictions to file
output = pd.DataFrame({'id': test_data.id,
                       'label': preds})
output.to_csv('submission2.csv', index=False)

In [109]:
test_data.head()

,id,gender,s11,s12,s13,s16,s17,s18,s48,s52,...,n6,n7,n8,n9,n10,n11,n12,n13,n14,n15
0,1,1,1,1,1,1,1,1,1,1,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1,1,1.000000,1
1,1,1,1,1,1,1,1,1,1,1,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1,1,1.000000,1
2,1,1,1,1,1,1,1,1,1,1,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1,1,1.000000,1
3,b'gAAAAABinOimitAnqlgOcqnD_LeNL3WEbXNGvjd3QVPi...,M,Y,Y,1,D,D,B,0,1,...,0.015684,-9.178200,1.675028,6.354938,5.427266,1.792147,0,0,0.003492,1
4,b'gAAAAABinOi3W9p3Oka5MV_dc2TeorZUcIWOnnODSx7E...,M,Y,Y,1,D,C,B,1,1,...,0.006406,-8.726633,1.963535,6.810158,11.487845,1.880992,0,0,0.060614,0


In [110]:
test_data = pd.read_csv("/content/drive/MyDrive/robi-datathon-2-pre-assessment/test.csv")
test_data.head()

,id,gender,s11,s12,s13,s16,s17,s18,s48,s52,...,n6,n7,n8,n9,n10,n11,n12,n13,n14,n15
0,b'gAAAAABinOi328DZcweGB4_nOyHA3Dy6o1YKYKyf3COx...,M,Y,Y,1,B,D,D,0,l,...,0.026301,-9.200175,2.122970,6.646617,4.801224,1.693087,0,0,0.439270,2
1,b'gAAAAABinOikutEIBjkUXl9lYTg4RI6jc4NfiMUCcVsn...,M,Y,Y,1,B,C,B,1,l,...,0.010774,-8.897092,1.770889,11.046294,6.407230,1.516728,0,0,0.084334,4
2,b'gAAAAABinOjBM70jBXOroAlUSq5lNXMd_oP0PU7jLQE5...,M,Y,N,1,B,D,B,1,l,...,0.016837,-9.173660,1.871872,2.805834,5.481219,1.788823,0,0,0.984210,0
3,b'gAAAAABinOimitAnqlgOcqnD_LeNL3WEbXNGvjd3QVPi...,M,Y,Y,1,D,D,B,0,1,...,0.015684,-9.178200,1.675028,6.354938,5.427266,1.792147,0,0,0.003492,1
4,b'gAAAAABinOi3W9p3Oka5MV_dc2TeorZUcIWOnnODSx7E...,M,Y,Y,1,D,C,B,1,1,...,0.006406,-8.726633,1.963535,6.810158,11.487845,1.880992,0,0,0.060614,0
